In [1]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import evaluate

/raid/nlp/sameer/anaconda3/envs/NL2bash/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tldr = load_dataset("neulab/tldr")

json_file_path = "/raid/nlp/sameer/NL2bash/flags.json"
context_file_path = "/raid/nlp/sameer/NL2bash/docprompting_data/tldr/manual_section.json"

with open(json_file_path, 'r') as j:
    flags_file = json.loads(j.read())

with open(context_file_path, 'r') as j:
    context_file = json.loads(j.read())

No config specified, defaulting to: tldr/data
Found cached dataset tldr (/raid/nlp/sameer/.cache/huggingface/datasets/neulab___tldr/data/1.1.0/a47b161300fcd808b86847f0c76f9de79519b07f20243aca13df0a5bed609a0b)
100%|██████████| 3/3 [00:00<00:00, 810.39it/s]


In [3]:
def add_context(data, tokenizer):
    
    nl_context = []
    for i in range(0,len(data)):
        j,context = 0,""
        while j < len(data[i]["oracle_man"]):
            if data[i]["oracle_man"][j] in context_file:
                new_context = "manual " + str(j) + ": " + context_file[data[i]["oracle_man"][j]]
                if len(tokenizer.encode(context + new_context)) < 2000:
                    context += new_context + "\n"
                else:
                    break
            j += 1
        context += 'query: ' + data[i]["nl"]
        nl_context.append(context)

    data = data.add_column("nl_context", nl_context)
    return(data)

In [4]:
def prepare_data():
    val_set = []
    test_set = []
    tldr_dataset = concatenate_datasets([tldr['train'], tldr['test']])
    val_set = tldr_dataset.filter(lambda example: example["cmd_name"] in cmd_flags)

    ds = val_set.train_test_split(test_size=0.25, seed=2)
    train_set = tldr_dataset.filter(lambda example: example["question_id"] not in ds["test"]["question_id"])
    test_set = tldr_dataset.filter(lambda example: example["question_id"] in ds["test"]["question_id"])
    test_set = test_set.filter(lambda example: example["cmd_name"] in cmd_flags)
    print("length of train set is: ", len(train_set))
    print("length of test set: ", len(test_set))
    return add_context(train_set, tokenizer_nl_context), add_context(test_set, tokenizer_nl_context)

In [2]:
tokenizer_nl_context = AutoTokenizer.from_pretrained("/raid/nlp/sameer/NL2bash/model_checkpoints/starcoder_base_1B_new_split")
model_nl_context = AutoModelForCausalLM.from_pretrained("/raid/nlp/sameer/NL2bash/model_checkpoints/starcoder_base_1B_new_split", device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
cmd_flags = [i.split(".")[0] for i in flags_file.keys()]

In [3]:
tldr = load_dataset("neulab/tldr")
tldr_dataset = concatenate_datasets([tldr['train'], tldr['test']])
tldr_dataset = tldr_dataset.train_test_split(test_size=0.15, seed=4)
tldr_dataset["test"] = tldr_dataset["test"].filter(lambda example: example["cmd_name"] in tldr_dataset["train"]["cmd_name"])
train_set, test_set = tldr_dataset["train"], tldr_dataset["test"]

No config specified, defaulting to: tldr/data
Found cached dataset tldr (/raid/nlp/sameer/.cache/huggingface/datasets/neulab___tldr/data/1.1.0/a47b161300fcd808b86847f0c76f9de79519b07f20243aca13df0a5bed609a0b)
100%|██████████| 3/3 [00:00<00:00, 813.53it/s]
Loading cached split indices for dataset at /raid/nlp/sameer/.cache/huggingface/datasets/neulab___tldr/data/1.1.0/a47b161300fcd808b86847f0c76f9de79519b07f20243aca13df0a5bed609a0b/cache-925aeb64c99ccc7b.arrow and /raid/nlp/sameer/.cache/huggingface/datasets/neulab___tldr/data/1.1.0/a47b161300fcd808b86847f0c76f9de79519b07f20243aca13df0a5bed609a0b/cache-74bcbc4d53757f8f.arrow
Loading cached processed dataset at /raid/nlp/sameer/.cache/huggingface/datasets/neulab___tldr/data/1.1.0/a47b161300fcd808b86847f0c76f9de79519b07f20243aca13df0a5bed609a0b/cache-d3ac47ec850a5142.arrow


In [6]:
for i in range(0,len(test_set)):
    if test_set[i]["cmd_name"] == "iperf":
        print(i)

851


In [11]:
f = open("train_tldr.txt", "a")
for i in range(0,len(train_set)):
    json_object = json.dumps(train_set[i], indent = 2) 

    f.write(json_object)
    f.write("\n")
    f.write("-----------------------------------------------------")
    f.write("\n")

In [4]:
cnt_nl = 0
cnt_flag = 0
for i in range(0,len(train_set)):
    flag = 0
    for token in train_set[i]["cmd"].split():
        if len(token) > 2 and token[0]=="-" and token[1]!="-":
            flag =1
            cnt_flag += 1
    if flag ==1:
        flag = 0
        cnt_nl += 1
print(cnt_nl, cnt_flag)
    

424 554


In [11]:
train_set, test_set = prepare_data()
outliers = [i["question_id"] for i in test_set if i["cmd_name"] not in train_set["cmd_name"]]
test_set = test_set.filter(lambda example: example["question_id"] not in outliers)

length of train set is:  6611
length of test set:  731


Filter: 100%|██████████| 731/731 [00:00<00:00, 18702.30 examples/s]


In [8]:
print(test_set[101]["nl_context"])

manual 0: runuser can be used to run commands with a substitute user and group ID. If the option -u is not given, runuser falls back to su-compatible semantics and a shell is executed. The difference between the commands runuser and su is that runuser does not ask for a password (because it may be executed by the root user only) and it uses a different PAM configuration. The command runuser does not have to be installed with set-user-ID permissions.
manual 1: -c, --command=command Pass command to the shell with the -c option.
manual 2: -g, --group=group The primary group to be used. This option is allowed for the root user only.
manual 3: -G, --supp-group=group Specify a supplementary group. This option is available to the root user only. The first specified supplementary group is also used as a primary group if the option --group is not specified.
manual 4: --session-command=command Same as -c, but do not create a new session. (Discouraged.)
query: run command as a different user and 

In [3]:
tokenizer_pre = AutoTokenizer.from_pretrained("/raid/nlp/sameer/NL2bash/model_checkpoints/starcoder_base_1B_new_split")
model_pre = AutoModelForCausalLM.from_pretrained("/raid/nlp/sameer/NL2bash/model_checkpoints/starcoder_base_1B_new_split").to("cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [122]:
flags_file["ethtool.txt"]

['--statistics',
 '--change-eeprom',
 '-p',
 '-S',
 '-L',
 '--negotiate',
 '--flash',
 '--show-offload',
 '--show-pause',
 '-X',
 '--set-eee',
 '--show-time-stamping',
 '--version',
 '-s',
 '--config-ntuple',
 '-Q',
 '-m',
 '-A',
 '--offload',
 '-U',
 '--show-ring',
 '-W',
 '--show-rxfh',
 '--module-info',
 '-N',
 '--show-priv-flags',
 '-d',
 '--get-dump',
 '--show-features',
 '-C',
 '--config-nfc',
 '--set-fec',
 '--show-coalesce',
 '--change',
 '--show-eee',
 '--set-dump',
 '-x',
 '-E',
 '--rxfh',
 '--show-rxfh-indir',
 '-c',
 '-n',
 '--per-queue',
 '--reset',
 '-r',
 '-u',
 '-w',
 '-a',
 '-h',
 '--phy-statistics',
 '--show-channels',
 '--set-ring',
 '-k',
 '--show-ntuple',
 '--identify',
 '-e',
 '--set-channels',
 '--register-dump',
 '-f',
 '--coalesce',
 '--get-phy-tunable',
 '-i',
 '-t',
 '-G',
 '-g',
 '--eeprom-dump',
 '--pause',
 '--set-priv-flags',
 '-K',
 '--test',
 '-P',
 '--features',
 '-l',
 '--show-fec',
 '--show-permaddr',
 '-T',
 '--show-nfc',
 '--set-rxfh-indir',
 '--du

In [8]:
def inference(model, tokenizer, sample):
    prompt = test_set[sample]["nl"] + tokenizer.sep_token
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda:0")
    attention_mask = inputs.attention_mask.to("cuda:0")
    gen_tokens = model.generate(
    input_ids,
    max_new_tokens=150,
    attention_mask = attention_mask
        )

    gen_tokens = gen_tokens.reshape(1, -1, gen_tokens.shape[-1])[0][0]
# to text and clean
    gen_code = tokenizer.decode(gen_tokens, skip_special_tokens=False)
    print(gen_code.split(tokenizer.sep_token)[-1])
    print(test_set[sample]["cmd"])

In [47]:
inference(model_nl_context, tokenizer_nl_context, 60)
# inference(model_pre, tokenizer_pre, 101)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


flashrom -w {{path/to/file}}<|endoftext|>
flashrom -p {{programmer}} --write {{path/to/file}}


In [7]:
f_cmd = open("pred_cmd_base_model.txt", "a")

In [5]:
# prompt = test_set[100]["nl_context"] + tokenizer.sep_token
from tqdm import tqdm
predictions = []
gt = []
for i in tqdm(range(0,len(test_set))):
    prompt = test_set[i]["nl"] + tokenizer_nl_context.sep_token
    inputs = tokenizer_nl_context(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda:4")
    attention_mask = inputs.attention_mask.to("cuda:4")
    gen_tokens = model_nl_context.generate(
    input_ids,
    # num_beams=5,
    # num_return_sequences=1,
    max_new_tokens=150,
    attention_mask = attention_mask
    )

    gen_tokens = gen_tokens.reshape(1, -1, gen_tokens.shape[-1])[0][0]
# to text and clean
    gen_code = tokenizer_nl_context.decode(gen_tokens, skip_special_tokens=False)
# gen_code = gen_code.split(tokenizer.sep_token)[2].strip().split(tokenizer.eos_token)[0].strip()
    predictions.append(gen_code.split(tokenizer_nl_context.sep_token)[1].strip().split(tokenizer_nl_context.eos_token)[0])
    # f_cmd.write(gen_code.split(tokenizer_nl_context.sep_token)[1].strip().split(tokenizer_nl_context.eos_token)[0] + "\n")
    gt.append(test_set["cmd"][i])


 42%|████▏     | 449/1081 [03:12<04:48,  2.19it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
 60%|█████▉    | 645/1081 [04:36<02:27,  2.95it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
 92%|█████████▏| 990/1081 [06:57<00:37,  2.43it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
100%|██████████| 1081/1081 [07:34<00:00,  2.38it/s]


In [25]:
len(predictions)

1081

In [10]:
import guidance
guidance_model = guidance.llms.Transformers(model = model_pre, tokenizer=tokenizer_pre)
guidance.llms.Transformers.cache.clear()

prompt = "create a reply template for a set of messages" + tokenizer_pre.sep_token + "notmuch" + """ --{{gen 'this' max_tokens=150}}"""
program = guidance(prompt)

executed_program = program(do_sample= True, llm = guidance_model, opt=['all', 'ignore', 'help', 'version'])
print(str(executed_program).split("<|sep|>")[1])
print(test_set[3]["cmd"])

NameError: name 'model_pre' is not defined

In [6]:
tldr_metric = evaluate.load('neulab/tldr_eval')
tldr_metric

EvaluationModule(name: "tldr_eval", module_type: "metric", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
predictions: list of str. The predictions
references: list of str. The references

Return
- **template_matching**: the exact match accuracy
- **command_accuracy**: accuracy of predicting the correct bash command name (e.g., `ls`)
- **bleu_char**: char bleu score
- **token recall/precision/f1**: the recall/precision/f1 of the predicted tokens
""", stored examples: 0)

In [17]:
print(test_set[0]["nl_context"].split("\n")[0].split(": ")[1] + "\n" + test_set[0]["nl_context"].split("\n")[-1].split(": ")[1])

apt-cache performs a variety of operations on APT's package cache. apt-cache does not manipulate the state of the system but does provide operations to search and generate interesting output from the package metadata. The metadata is acquired and updated via the 'update' command of e.g.  apt-get, so that it can be outdated if the last update is too long ago, but in exchange apt-cache works independently of the availability of the configured sources (e.g. offline).
show whether a package is installed and up to date


In [14]:
import guidance

guidance_model = guidance.llms.Transformers(model = model_nl_context, tokenizer=tokenizer_nl_context, device=0)
guidance.llms.Transformers.cache.clear()

predictions = []
gt = []
for i in range(0,len(test_set)):
    prompt = test_set[i]["nl"] + tokenizer_nl_context.sep_token + """{{gen 'this' max_tokens=150}}"""
    program = guidance(prompt)

    executed_program = program(do_sample= True, llm = guidance_model)
    predictions.append(str(executed_program).split("<|sep|>")[1])
    gt.append(test_set[i]["cmd"])

Stop program decompress a file and write to stdout<|sep|> xz -dc {{file.xz}}

In [11]:
predictions[1], test_set["cmd"][1]

('bosh -e {{environment}} get-cloud-config<|endoftext|>',
 'bosh -e {{environment}} cloud-config')

In [7]:
predictions = [i.split(tokenizer_nl_context.eos_token)[0] for i in predictions]

In [ ]:
tldr_metric.compute(predictions=predictions, references=gt) #pretrained model with nl context

{'template_matching': 0.20166512488436633,
 'command_accuracy': 0.5393154486586494,
 'bleu_char': 51.49728869534558,
 'token_recall': 0.5556052642818827,
 'token_precision': 0.5759678884235094,
 'token_f1': 0.5546365924090315}

In [27]:
prompt = test_set["nl"][8] + tokenizer_nl_context.sep_token
inputs = tokenizer_nl_context(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda:0")
attention_mask = inputs.attention_mask.to("cuda:0")
gen_tokens = model_nl_context.generate(
    input_ids,
    max_new_tokens=150,
    attention_mask = attention_mask
    )

gen_tokens = gen_tokens.reshape(1, -1, gen_tokens.shape[-1])[0][0]
# to text and clean
gen_code = tokenizer_nl_context.decode(gen_tokens, skip_special_tokens=False)
# gen_code = gen_code.split(tokenizer.sep_token)[2].strip().split(tokenizer.eos_token)[0].strip()
print(gen_code.split(tokenizer_nl_context.sep_token)[1])
# gt.append(test_set["cmd"][0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 drill {{example.com}}</s>


In [26]:
test_set["nl"][8], test_set["cmd_name"][8], test_set["cmd"][8]

('perform a reverse dns lookup', 'getent', 'getent hosts {{host}}')